# SoniTranslate

| Description | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/SoniTranslate/) |
| 🚀 Online Demo in HF | [![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces/r3gm/SoniTranslate_translate_audio_of_a_video_content) |




In [1]:
#@title [STEP 01/08] Repo owners & refs — CONFIG ONLY
#@markdown Конфигурация источников. Можно переопределить через ENV переменные.

OWNER = {"original": "R3gm", "fork": "tekswirl25"}
USE = "fork"  #@param ["original", "fork"]

REPOS = {
    "sonitranslate": {"name": "SoniTranslate",   "ref": "main"},
    "whisperx":      {"name": "whisperX",        "ref": "cuda_12_x"},
    "pyannote":      {"name": "pyannote-audio",  "ref": "3.1.1"},
}

import os, json

for key, info in REPOS.items():
    PREFIX = key.upper()
    if not os.environ.get(f"{PREFIX}_URL"):
        os.environ[f"{PREFIX}_URL"] = f"https://github.com/{OWNER[USE]}/{info['name']}.git"
    if not os.environ.get(f"{PREFIX}_REF"):
        os.environ[f"{PREFIX}_REF"] = info["ref"]
    # совместимость: BRANCH алиас к REF
    os.environ[f"{PREFIX}_BRANCH"] = os.environ[f"{PREFIX}_REF"]

print("MODE:", USE, "| OWNER:", OWNER[USE])
print(json.dumps({
    "SONITRANSLATE": [os.environ["SONITRANSLATE_URL"], os.environ["SONITRANSLATE_REF"]],
    "WHISPERX":      [os.environ["WHISPERX_URL"],      os.environ["WHISPERX_REF"]],
    "PYANNOTE":      [os.environ["PYANNOTE_URL"],      os.environ["PYANNOTE_REF"]],
}, indent=2, ensure_ascii=False))


MODE: fork | OWNER: tekswirl25
{
  "SONITRANSLATE": [
    "https://github.com/tekswirl25/SoniTranslate.git",
    "main"
  ],
  "WHISPERX": [
    "https://github.com/tekswirl25/whisperX.git",
    "cuda_12_x"
  ],
  "PYANNOTE": [
    "https://github.com/tekswirl25/pyannote-audio.git",
    "3.1.1"
  ]
}


In [2]:
%%bash
# [STEP 02/08] Quick echo — human check (idempotent)
set -euo pipefail
LOG_DIR="${LOG_DIR:-/content/_install_logs}"
mkdir -p "$LOG_DIR"

{
  echo "SONITRANSLATE: ${SONITRANSLATE_URL:-unset} @ ${SONITRANSLATE_REF:-unset}"
  echo "WHISPERX:      ${WHISPERX_URL:-unset}      @ ${WHISPERX_REF:-unset}"
  echo "PYANNOTE:      ${PYANNOTE_URL:-unset}      @ ${PYANNOTE_REF:-unset}"
} | tee "$LOG_DIR/02_echo.txt"


SONITRANSLATE: https://github.com/tekswirl25/SoniTranslate.git @ main
WHISPERX:      https://github.com/tekswirl25/whisperX.git      @ cuda_12_x
PYANNOTE:      https://github.com/tekswirl25/pyannote-audio.git      @ 3.1.1


In [3]:
%%bash
# [STEP 03/08] Remote ref validation — branch/tag auto-detect, ENV-compatible
set -euo pipefail
LOG_DIR="${LOG_DIR:-/content/_install_logs}"
mkdir -p "$LOG_DIR"

check_ref () {
  local url="$1" ref="$2" type="${3:-}"
  local name
  name="$(basename "$url" .git)"

  # Helper checks
  local tag_ok=1 head_ok=1

  if [[ -n "$type" ]]; then
    # Respect explicit type if provided
    if [[ "$type" == "tag" ]]; then
      if git ls-remote --tags "$url" "refs/tags/${ref}" >/dev/null 2>&1; then
        echo "[OK] ${name}: tag '${ref}' found"
      else
        echo "[FAIL] ${name}: tag '${ref}' NOT found"
      fi
    else
      if git ls-remote --heads "$url" "$ref" >/dev/null 2>&1; then
        echo "[OK] ${name}: branch '${ref}' found"
      else
        echo "[FAIL] ${name}: branch '${ref}' NOT found"
      fi
    fi
    return 0
  fi

  # Auto-detect when REFTYPE is not provided
  git ls-remote --tags  "$url" "refs/tags/${ref}" >/dev/null 2>&1 && tag_ok=0 || tag_ok=1
  git ls-remote --heads "$url" "$ref"              >/dev/null 2>&1 && head_ok=0 || head_ok=1

  if [[ $tag_ok -eq 0 && $head_ok -eq 0 ]]; then
    echo "[OK] ${name}: ref '${ref}' exists as BOTH (tag & branch)"
  elif [[ $tag_ok -eq 0 ]]; then
    echo "[OK] ${name}: tag '${ref}' found"
  elif [[ $head_ok -eq 0 ]]; then
    echo "[OK] ${name}: branch '${ref}' found"
  else
    echo "[FAIL] ${name}: ref '${ref}' not found as tag or branch"
  fi
}

{
  check_ref "${SONITRANSLATE_URL:-}" "${SONITRANSLATE_REF:-}" "${SONITRANSLATE_REFTYPE:-}"
  check_ref "${WHISPERX_URL:-}"      "${WHISPERX_REF:-}"      "${WHISPERX_REFTYPE:-}"
  check_ref "${PYANNOTE_URL:-}"      "${PYANNOTE_REF:-}"      "${PYANNOTE_REFTYPE:-}"
} | tee "$LOG_DIR/03_validate_refs.txt"

[OK] SoniTranslate: ref 'main' exists as BOTH (tag & branch)
[OK] whisperX: ref 'cuda_12_x' exists as BOTH (tag & branch)
[OK] pyannote-audio: ref '3.1.1' exists as BOTH (tag & branch)


In [4]:
%%bash
# [STEP 04/08] SAFE CLONE & REQUIREMENTS PREVIEW (NO INSTALL)
set -euo pipefail
LOG_DIR="${LOG_DIR:-/content/_install_logs}"
mkdir -p "$LOG_DIR"

REPO_DIR="/content/SoniTranslate_debug"
URL="${SONITRANSLATE_URL:-}"
REF="${SONITRANSLATE_REF:-}"
REQ="requirements_base.txt"

{
  echo "== repo: $URL @ $REF =="

  # fresh shallow clone
  rm -rf "$REPO_DIR"
  git clone --depth=2 "$URL" "$REPO_DIR" -q
  cd "$REPO_DIR"

  # fetch + checkout supports both tag and branch safely
  git fetch --depth=2 origin "$REF" -q || true
  if git rev-parse --verify -q "refs/remotes/origin/$REF" >/dev/null; then
    git checkout -qf "origin/$REF"
  elif git rev-parse --verify -q "refs/tags/$REF" >/dev/null; then
    git checkout -qf "refs/tags/$REF"
  else
    echo "[WARN] ref '$REF' not found as branch or tag; staying on default clone HEAD"
  fi

  if [[ ! -f "$REQ" ]]; then
    echo "[info] '$REQ' not found — nothing to preview."
    exit 0
  fi

  echo "== current whisperX lines in ${REQ} =="
  grep -n -E 'git\+https://github\.com/.*/whisperX\.git@.*' "$REQ" || echo "[info] whisperX line not found"

  echo "== current pyannote lines in ${REQ} =="
  grep -n -E 'git\+https://github\.com/.*/pyannote-audio\.git@.*' "$REQ" || echo "[info] pyannote line not found"

  cp "$REQ" "${REQ}.preview"

  # Substitute to ENV-refs in preview only (no deps install here)
  sed -i "s|git+https://github.com/.*/whisperX.git@.*|git+${WHISPERX_URL:-https://github.com/placeholder/whisperX.git}@${WHISPERX_REF:-main}|" "${REQ}.preview"
  sed -i "s|git+https://github.com/.*/pyannote-audio.git@.*|git+${PYANNOTE_URL:-https://github.com/placeholder/pyannote-audio.git}@${PYANNOTE_REF:-3.1.1}|" "${REQ}.preview"

  echo "== PREVIEW DIFF (original vs preview) =="
  diff -u "$REQ" "${REQ}.preview" || true

  rm -f "${REQ}.preview"
  echo "[done] preview only; original requirements not modified."
} | tee "$LOG_DIR/04_clone_preview.txt"



== repo: https://github.com/tekswirl25/SoniTranslate.git @ main ==
== current whisperX lines in requirements_base.txt ==
12:git+https://github.com/R3gm/whisperX.git@cuda_11_8
== current pyannote lines in requirements_base.txt ==
11:git+https://github.com/R3gm/pyannote-audio.git@3.1.1
== PREVIEW DIFF (original vs preview) ==
--- requirements_base.txt	2025-08-27 06:35:54.640716992 +0000
+++ requirements_base.txt.preview	2025-08-27 06:35:55.314764506 +0000
@@ -8,8 +8,8 @@
 pydub==0.25.1
 edge_tts==6.1.7
 deep_translator==1.11.4
-git+https://github.com/R3gm/pyannote-audio.git@3.1.1
-git+https://github.com/R3gm/whisperX.git@cuda_11_8
+git+https://github.com/tekswirl25/pyannote-audio.git@3.1.1
+git+https://github.com/tekswirl25/whisperX.git@cuda_12_x
 nest_asyncio
 gTTS
 gradio_client==0.10.1
[done] preview only; original requirements not modified.


In [5]:
%%bash
# [STEP 05/08] SCAN REQUIREMENTS (READ-ONLY AUDIT)
set -euo pipefail

rm -rf SoniTranslate_scan
git clone -q --depth=2 "${SONITRANSLATE_URL}" SoniTranslate_scan
cd SoniTranslate_scan
git fetch -q --depth=2 origin "${SONITRANSLATE_REF}" || true
git checkout -qf "origin/${SONITRANSLATE_REF}" 2>/dev/null || git checkout -qf "refs/tags/${SONITRANSLATE_REF}" 2>/dev/null || echo "[WARN] ref not found; using default HEAD"

echo "== FILES =="
find . -maxdepth 2 -type f -name "requirements*.txt" -printf "%P\n" | sort || true

echo -e "\n== GREP: torch with +cu suffix =="
grep -nE '^torch[^#]*\+cu[0-9_]+' requirements*.txt */requirements*.txt 2>/dev/null || echo "[ok] no '+cu' torch pins found"

echo -e "\n== GREP: TTS==0.21.1 =="
grep -nE '(^|[^A-Za-z])TTS==0\.21\.1([^A-Za-z]|$)' requirements*.txt */requirements*.txt 2>/dev/null || echo "[ok] no TTS==0.21.1 pins"

echo -e "\n== GREP: whisperX git lines =="
grep -nE 'git\+https://github\.com/.*/whisperX\.git@.*' requirements*.txt */requirements*.txt 2>/dev/null || echo "[info] no whisperX git lines found"

echo -e "\n== GREP: websockets/opencv hard pins (for awareness) =="
grep -nE 'websockets|opencv-python' requirements*.txt */requirements*.txt 2>/dev/null || echo "[info] none"



== FILES ==
requirements_base.txt
requirements_extra.txt
requirements.txt
requirements_xtts.txt

== GREP: torch with +cu suffix ==
requirements_base.txt:2:torch>=2.1.0+cu118
requirements_base.txt:3:torchvision>=0.16.0+cu118
requirements_base.txt:4:torchaudio>=2.1.0+cu118
requirements.txt:4:torch>=2.1.0+cu118
requirements.txt:5:torchvision>=0.16.0+cu118
requirements.txt:6:torchaudio>=2.1.0+cu118
[ok] no '+cu' torch pins found

== GREP: TTS==0.21.1 ==
requirements_xtts.txt:58:# pip install TTS==0.21.1  --no-deps
[ok] no TTS==0.21.1 pins

== GREP: whisperX git lines ==
requirements_base.txt:12:git+https://github.com/R3gm/whisperX.git@cuda_11_8
requirements.txt:12:git+https://github.com/m-bain/whisperX.git@a5dca2c
[info] no whisperX git lines found

== GREP: websockets/opencv hard pins (for awareness) ==
[info] none


In [6]:
%%bash
# [STEP 06/08] PREVIEW FIXES (NO WRITE)
set -euo pipefail

rm -rf SoniTranslate_fixpreview
git clone -q --depth=2 "${SONITRANSLATE_URL}" SoniTranslate_fixpreview
cd SoniTranslate_fixpreview
git fetch --depth=2 origin "${SONITRANSLATE_REF}" -q || true
git checkout -qf "origin/${SONITRANSLATE_REF}" 2>/dev/null || git checkout -qf "refs/tags/${SONITRANSLATE_REF}" 2>/dev/null || echo "[WARN] ref not found; using default HEAD"

fix_one() {
  local file="$1"
  [[ -f "$file" ]] || return 0
  cp "$file" "${file}.preview"

  # 1) torch с суффиксом +cu... → заменить на просто 'torch'
  sed -i -E 's/^torch[^#]*\+cu[0-9_]+/torch/' "${file}.preview"

  # 2) TTS==0.21.1 → диапазон для colab/python3.12
  sed -i -E 's/(^|[^A-Za-z])TTS==0\.21\.1([^A-Za-z]|$)/TTS>=0.22,<0.23/g' "${file}.preview"

  echo "### DIFF for $file"
  diff -u "$file" "${file}.preview" || true
  rm -f "${file}.preview"
}

for f in requirements*.txt; do fix_one "$f"; done
echo "[done] only preview; no files modified."



### DIFF for requirements_base.txt
--- requirements_base.txt	2025-08-27 06:35:59.519060870 +0000
+++ requirements_base.txt.preview	2025-08-27 06:36:00.268113671 +0000
@@ -1,7 +1,7 @@
 --extra-index-url https://download.pytorch.org/whl/cu118
-torch>=2.1.0+cu118
-torchvision>=0.16.0+cu118
-torchaudio>=2.1.0+cu118
+torch
+torch
+torch
 ctranslate2<=4.4.0
 yt-dlp
 gradio==4.19.2
### DIFF for requirements_extra.txt
### DIFF for requirements.txt
--- requirements.txt	2025-08-27 06:35:59.519060870 +0000
+++ requirements.txt.preview	2025-08-27 06:36:00.288115081 +0000
@@ -1,9 +1,9 @@
 # Temporal requirements
 nest_asyncio
 --extra-index-url https://download.pytorch.org/whl/cu118
-torch>=2.1.0+cu118
-torchvision>=0.16.0+cu118
-torchaudio>=2.1.0+cu118
+torch
+torch
+torch
 yt-dlp
 gradio==4.19.2
 pydub==0.25.1
### DIFF for requirements_xtts.txt
--- requirements_xtts.txt	2025-08-27 06:35:59.519060870 +0000
+++ requirements_xtts.txt.preview	2025-08-27 06:36:00.298115786 +0000
@@ -55,4 +55,4 @@
 
 #

In [7]:
%%bash
# [STEP 07/08] CREATE CONSTRAINTS.TXT (READABLE PINS)
cat > /content/constraints_sonitranslate.txt <<'TXT'
# soft constraints to avoid common conflicts (used with --constraint)
# torch ставим отдельно до requirements, поэтому здесь его нет

scipy>=1.11
websockets>=15,<16
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.41
sentence-transformers>=3.0
gradio
TXT

echo "== constraints_sonitranslate.txt =="
cat /content/constraints_sonitranslate.txt


== constraints_sonitranslate.txt ==
# soft constraints to avoid common conflicts (used with --constraint)
# torch ставим отдельно до requirements, поэтому здесь его нет

scipy>=1.11
websockets>=15,<16
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.41
sentence-transformers>=3.0
gradio


In [8]:
#@title [STEP 07MODE/08] NUMPY ↔ GRADIO PROFILE (CONFIG ONLY)
#@markdown Профили:
#@markdown • **upstream** — как в оригинале (gradio 4.19.2 → numpy 1.26.4)
#@markdown • **modern** — эксперимент c NumPy 2.x (gradio ≥ 4.30)
PROFILE = "upstream"  #@param ["upstream", "modern"]

if PROFILE == "upstream":
    NUMPY_SPEC  = "numpy==1.26.4"
    GRADIO_SPEC = "gradio==4.19.2"
elif PROFILE == "modern":
    NUMPY_SPEC  = "numpy>=2.1,<2.3"
    GRADIO_SPEC = "gradio>=4.30"
else:
    raise ValueError("PROFILE must be 'upstream' or 'modern'")

import os, json
# уважаем внешние overrides, если они уже выставлены
os.environ.setdefault("NUMPY_SPEC",  NUMPY_SPEC)
os.environ.setdefault("GRADIO_SPEC", GRADIO_SPEC)
os.environ.setdefault("PROFILE",     PROFILE)

print("PROFILE:", os.environ["PROFILE"])
print(json.dumps({
    "NUMPY_SPEC":  os.environ["NUMPY_SPEC"],
    "GRADIO_SPEC": os.environ["GRADIO_SPEC"]
}, indent=2))



PROFILE: upstream
{
  "NUMPY_SPEC": "numpy==1.26.4",
  "GRADIO_SPEC": "gradio==4.19.2"
}


In [9]:
# [STEP 07AUTO/08] PY VERSION AWARE NUMPY/GRADIO (CONFIG ONLY, NON-OVERRIDING)
import sys, os, json, re

py = sys.version_info
py_str = f"{py.major}.{py.minor}.{py.micro}"
print("Detected Python:", py_str)

# Берём то, что уже задал профиль (07MODE). Ничего не перезаписываем.
NUMPY_SPEC  = os.environ.get("NUMPY_SPEC")   # может быть None
GRADIO_SPEC = os.environ.get("GRADIO_SPEC")  # может быть None

# Если профиль ничего не задал — подберём дефолты под версию Python.
# Примечание: NumPy 2.x поддерживает Python ≥3.9, поэтому 3.12 ОК и для 1.26.4, и для 2.x.
if not NUMPY_SPEC:
    if (py.major, py.minor) >= (3, 11):
        # современный дефолт; безопасен для 3.11+ и не конфликтует с большинством стеков
        NUMPY_SPEC = "numpy>=2.1,<2.3"
    else:
        NUMPY_SPEC = "numpy==1.26.4"

if not GRADIO_SPEC:
    # дефолт «как в апстриме», если профиль его не задал
    GRADIO_SPEC = "gradio==4.19.2"

# Лёгкая диагностика потенциальных несостыковок (только предупреждаем)
def major_ver(spec: str, name: str):
    if not spec: return None
    m = re.search(rf"{name}\s*([<>=!~]=\s*)?(\d+)\.(\d+)", spec.replace(" ", ""), re.I)
    return int(m.group(2)) if m else None

np_major = major_ver(NUMPY_SPEC, "numpy")
gr_set   = bool(GRADIO_SPEC)

if np_major == 2 and GRADIO_SPEC == "gradio==4.19.2":
    print("[WARN] NumPy 2.x с gradio==4.19.2: если поймаете конфликт, поднимите gradio (например, '>=4.30') или верните NumPy 1.26.4.")

# Экспорт без перезаписи ранее заданных — используем уже рассчитанные значения
os.environ["NUMPY_SPEC"]  = NUMPY_SPEC
os.environ["GRADIO_SPEC"] = GRADIO_SPEC

print(json.dumps({"Python": py_str, "NUMPY_SPEC": NUMPY_SPEC, "GRADIO_SPEC": GRADIO_SPEC}, indent=2))



Detected Python: 3.12.11
{
  "Python": "3.12.11",
  "NUMPY_SPEC": "numpy==1.26.4",
  "GRADIO_SPEC": "gradio==4.19.2"
}


In [17]:
# [STEP 07DBG/08] DEBUG PROFILE — manual toggles for secondary deps
import os, json

# === двойные строки: можно раскомментировать хардкод и закомментировать ENV ===
# SCIPY_SPEC        = os.environ.get("SCIPY_SPEC")
SCIPY_SPEC        = "scipy>=1.10"

# WEBSOCKETS_SPEC   = os.environ.get("WEBSOCKETS_SPEC")
#WEBSOCKETS_SPEC   = "websockets>=15,<16"
WEBSOCKETS_SPEC   = "websockets>=10,<12"

# OPENCV_SPEC       = os.environ.get("OPENCV_SPEC")
OPENCV_SPEC       = "opencv-python==4.10.0.84"

# TTS_SPEC          = os.environ.get("TTS_SPEC")
TTS_SPEC          = "TTS>=0.22,<0.23"

# TRANSFORMERS_SPEC = os.environ.get("TRANSFORMERS_SPEC")
TRANSFORMERS_SPEC = "transformers>=4.41"

#SENT_TR_SPEC      = os.environ.get("SENTENCE_TRANSFORMERS_SPEC")
SENT_TR_SPEC      = "sentence-transformers>=3.0"

# === TORCH trio (torch/torchvision/torchaudio) ===
#TORCH_SPEC       = os.environ.get("TORCH_SPEC")
TORCH_SPEC       = "torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1"

#TORCH_INDEX_URL  = os.environ.get("TORCH_INDEX_URL")
TORCH_INDEX_URL  = "https://download.pytorch.org/whl/cpu"
TORCH_INDEX_URL  = "https://download.pytorch.org/whl/cu121"   # пример для CUDA 12.1


def set_if_nonempty(key, val):
    """Write to ENV only if val is a non-empty string; otherwise leave as-is."""
    if isinstance(val, str) and val.strip():
        os.environ[key] = val.strip()

# === записываем выбранные в ENV (только непустые строки) ===
set_if_nonempty("SCIPY_SPEC",        SCIPY_SPEC)
set_if_nonempty("WEBSOCKETS_SPEC",   WEBSOCKETS_SPEC)
set_if_nonempty("OPENCV_SPEC",       OPENCV_SPEC)
set_if_nonempty("TTS_SPEC",          TTS_SPEC)
set_if_nonempty("TRANSFORMERS_SPEC", TRANSFORMERS_SPEC)
set_if_nonempty("SENTENCE_TRANSFORMERS_SPEC", SENT_TR_SPEC)
set_if_nonempty("TORCH_SPEC", TORCH_SPEC)
set_if_nonempty("TORCH_INDEX_URL", TORCH_INDEX_URL)


print("== DEBUG PROFILE (effective) ==")
print(json.dumps({
    "NUMPY":  os.environ.get("NUMPY_SPEC"),    # управляется только через 07MODE/07AUTO
    "GRADIO": os.environ.get("GRADIO_SPEC"),   # управляется только через 07MODE/07AUTO
    "SCIPY":  os.environ.get("SCIPY_SPEC"),
    "WEBSOCKETS": os.environ.get("WEBSOCKETS_SPEC"),
    "OPENCV": os.environ.get("OPENCV_SPEC"),
    "TTS": os.environ.get("TTS_SPEC"),
    "TRANSFORMERS": os.environ.get("TRANSFORMERS_SPEC"),
    "SENTENCE-TRANSFORMERS": os.environ.get("SENTENCE_TRANSFORMERS_SPEC"),
    "TORCH_SPEC": os.environ.get("TORCH_SPEC"),
    "TORCH_INDEX_URL": os.environ.get("TORCH_INDEX_URL"),

}, indent=2))





== DEBUG PROFILE (effective) ==
{
  "NUMPY": "numpy==1.26.4",
  "GRADIO": "gradio==4.19.2",
  "SCIPY": "scipy>=1.10",
  "WEBSOCKETS": "websockets>=10,<12",
  "OPENCV": "opencv-python==4.10.0.84",
  "TTS": "TTS>=0.22,<0.23",
  "TRANSFORMERS": "transformers>=4.41",
  "SENTENCE-TRANSFORMERS": "sentence-transformers>=3.0",
  "TORCH_SPEC": "torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1",
  "TORCH_INDEX_URL": "https://download.pytorch.org/whl/cu121"
}


In [19]:
%%bash
# [STEP 07CONS/08] WRITE CONSTRAINTS FROM ENV (STRICT FOR NUMPY/GRADIO)
set -euo pipefail

# обязательные значения: приходят из 07MODE/07AUTO (или 07DBG)
: "${NUMPY_SPEC?NUMPY_SPEC is not set. Run STEP 07MODE/07AUTO first.}"
: "${GRADIO_SPEC?GRADIO_SPEC is not set. Run STEP 07MODE/07AUTO first.}"

# опциональные оверрайды (могут прийти из 07DBG); иначе — разумные дефолты
: "${SCIPY_SPEC:=scipy>=1.10}"
: "${WEBSOCKETS_SPEC:=websockets>=15,<16}"
: "${OPENCV_SPEC:=opencv-python==4.10.0.84}"
: "${TTS_SPEC:=TTS>=0.22,<0.23}"
: "${TRANSFORMERS_SPEC:=transformers>=4.41}"
: "${SENTENCE_TRANSFORMERS_SPEC:=sentence-transformers>=3.0}"

CONS="/content/constraints_sonitranslate.txt"
mkdir -p /content

cat > "$CONS" <<TXT
${NUMPY_SPEC}
${SCIPY_SPEC}
${WEBSOCKETS_SPEC}
${OPENCV_SPEC}
${TTS_SPEC}
${TRANSFORMERS_SPEC}
${SENTENCE_TRANSFORMERS_SPEC}
${GRADIO_SPEC}
TXT

echo "== USING CONSTRAINTS (PROFILE=${PROFILE:-unknown}) =="
cat "$CONS"



== USING CONSTRAINTS (PROFILE=upstream) ==
numpy==1.26.4
scipy>=1.10
websockets>=10,<12
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.41
sentence-transformers>=3.0
gradio==4.19.2


In [20]:
%%bash
# [STEP 08A/08] PRECHECK UV & GIT (NO INSTALL)
set -euo pipefail

echo "== python/pip/uv/git versions =="
python -V
python -m pip -V
git --version || true
python -c "import shutil; print('uv on PATH:', bool(shutil.which('uv')))"

echo "== installing uv if missing =="
python -m pip install -q --upgrade pip setuptools wheel
python -m pip install -q uv
python -c "import shutil; print('uv on PATH (after):', bool(shutil.which('uv')))"

echo "== env =="
echo "SONITRANSLATE_URL = ${SONITRANSLATE_URL}"
echo "SONITRANSLATE_REF = ${SONITRANSLATE_REF}"
echo "WHISPERX_URL      = ${WHISPERX_URL}"
echo "WHISPERX_REF      = ${WHISPERX_REF}"

echo "== remote ref check =="
git ls-remote --tags  "${SONITRANSLATE_URL}" "${SONITRANSLATE_REF}" || git ls-remote --heads "${SONITRANSLATE_URL}" "${SONITRANSLATE_REF}" || echo "[warn] ref not found"
git ls-remote --tags  "${WHISPERX_URL}"      "${WHISPERX_REF}"      || git ls-remote --heads "${WHISPERX_URL}"      "${WHISPERX_REF}"      || echo "[warn] ref not found"

echo "[ok] precheck done"


== python/pip/uv/git versions ==
Python 3.12.11
pip 25.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
git version 2.34.1
uv on PATH: True
== installing uv if missing ==
uv on PATH (after): True
== env ==
SONITRANSLATE_URL = https://github.com/tekswirl25/SoniTranslate.git
SONITRANSLATE_REF = main
WHISPERX_URL      = https://github.com/tekswirl25/whisperX.git
WHISPERX_REF      = cuda_12_x
== remote ref check ==
[ok] precheck done


In [21]:
%%bash
# [STEP 08AA/08] QUICK FIX: satisfy IPython 7.x -> jedi>=0.16
set -euo pipefail

# Разрешим ненулевой код у python-блока, чтобы поймать его вручную:
set +e
python - <<'PY'
import sys
try:
    import IPython
    ver = getattr(IPython, "__version__", "0")
    major = int(ver.split(".")[0])
    if major == 7:
        try:
            import jedi  # ok
            print("IPython 7.x: jedi present -> skip install")
            sys.exit(0)
        except Exception:
            print("IPython 7.x: jedi missing -> need install")
            sys.exit(42)
    else:
        print(f"IPython {ver}: not 7.x -> skip install")
        sys.exit(0)
except Exception as e:
    print(f"[warn] IPython check failed: {e}")
    sys.exit(0)
PY
rc=$?
set -e

if [[ $rc -eq 42 ]]; then
  python -m pip install -q "jedi>=0.16"
  echo "[ok] installed jedi (for IPython 7.x)"
else
  echo "[skip] no jedi install needed"
fi



IPython 7.x: jedi present -> skip install
[skip] no jedi install needed


In [22]:
%%bash
# [STEP 08/08] DRY INSTALL ON CPU (ISOLATED COPY, VERBOSE LOGS)
# profile-driven; pins только из 07MODE/07AUTO/07CONS; без хардкодов здесь
set -euo pipefail

LOG_DIR="/content/_install_logs"
mkdir -p "$LOG_DIR"

echo "Python: $(python -V)"
echo "PIP_CONSTRAINT=${PIP_CONSTRAINT:-<unset>}"
echo "PROFILE=${PROFILE:-<unset>}"
echo "NUMPY_SPEC=${NUMPY_SPEC:-<unset>}"
echo "GRADIO_SPEC=${GRADIO_SPEC:-<unset>}"
echo "=== ACTIVE CONSTRAINTS FILE ==="
test -s /content/constraints_sonitranslate.txt || { echo "ERROR: /content/constraints_sonitranslate.txt missing"; exit 2; }
sed -n '1,120p' /content/constraints_sonitranslate.txt

export PIP_DISABLE_PIP_VERSION_CHECK=1
export PIP_PROGRESS_BAR=on
export PIP_USE_PEP517=1
export PIP_PREFER_BINARY=1
export PYTHONUNBUFFERED=1   # <— важное: немедленный вывод из python

on_fail() {
  echo ""
  echo "===== INSTALL FAILED — LAST 200 LINES OF LOGS ====="
  shopt -s nullglob
  for f in "$LOG_DIR"/*.log; do
    echo "--- $(basename "$f") ---"
    tail -n 200 "$f" || true
  done
  echo "===== DIR TREE ====="
  (set +e; ls -R . | sed 's/^/    /')
}
trap on_fail ERR

# Потоковый запуск с метками времени (HH:MM:SS) и построчной буферизацией
run() {
  echo -e "\n+ $*" | tee -a "$LOG_DIR/commands.log"
  (stdbuf -oL -eL bash -c "$*") 2>&1 \
    | awk '{ cmd="date +[%H:%M:%S]"; cmd | getline t; close(cmd); print t, $0; fflush(); }' \
    | tee -a "$LOG_DIR/combined.log"
}

echo "== ENV =="
echo "SONITRANSLATE: ${SONITRANSLATE_URL} @ ${SONITRANSLATE_REF}"
echo "WHISPERX:      ${WHISPERX_URL} @ ${WHISPERX_REF}"
echo "PYANNOTE:      ${PYANNOTE_URL} @ ${PYANNOTE_REF}"
echo "LOG_DIR:       ${LOG_DIR}"

# 0) bootstrap
run "python -m pip install -v --upgrade pip wheel uv --log $LOG_DIR/pip_bootstrap.log"
run "python -m pip install -v 'setuptools<81' --log $LOG_DIR/pip_setuptools_compat.log"

# 1) pre-pin numpy + gradio (без внешних constraints; gradio — без зависимостей)
if [[ -z "${NUMPY_SPEC:-}" || -z "${GRADIO_SPEC:-}" ]]; then
  echo "ERROR: NUMPY_SPEC/GRADIO_SPEC not set (run 07MODE/07AUTO/07CONS)."
  exit 3
fi
run "env -u PIP_CONSTRAINT uv run python -m pip install -v '${NUMPY_SPEC}'  --log $LOG_DIR/pip_prepin_numpy.log"
run "env -u PIP_CONSTRAINT uv run python -m pip install -v '${GRADIO_SPEC}' --no-deps --log $LOG_DIR/pip_prepin_gradio.log"
# (опц.) чисто-питоновый docopt
run "env -u PIP_ONLY_BINARY -u PIP_CONSTRAINT uv run python -m pip install -v 'docopt>=0.6.2' --log $LOG_DIR/pip_docopt.log || true"

# 2) CPU/GPU torch-тройка без жёстких версий; задавай в 07DBG при необходимости
TORCH_INDEX_URL="${TORCH_INDEX_URL:-https://download.pytorch.org/whl/cpu}"
TORCH_SPEC="${TORCH_SPEC:-torch torchvision torchaudio}"
run "uv run python -m pip install -v --index-url '${TORCH_INDEX_URL}' ${TORCH_SPEC} --log $LOG_DIR/pip_torch_cpu.log"

# 3) чистая копия для установки (branch/tag-safe checkout)
run "rm -rf SoniTranslate_installtest"
run "git clone -q --depth=2 '${SONITRANSLATE_URL}' SoniTranslate_installtest"
cd SoniTranslate_installtest
git fetch -q --depth=2 origin "${SONITRANSLATE_REF}" || true
git checkout -qf "origin/${SONITRANSLATE_REF}" 2>/dev/null || git checkout -qf "refs/tags/${SONITRANSLATE_REF}" 2>/dev/null || echo "[WARN] ref not found; using default HEAD"

# 4) локальные правки ТОЛЬКО в копии requirements
run "sed -i \"s|git+https://github.com/.*/whisperX.git@.*|git+${WHISPERX_URL}@${WHISPERX_REF}|\" requirements_base.txt"
run "sed -i \"s|git+https://github.com/.*/pyannote-audio.git@.*|git+${PYANNOTE_URL}@${PYANNOTE_REF}|\" requirements_base.txt"
run "sed -i -E 's/^torch[^#]*\\+cu[0-9_]+/torch/' requirements_base.txt"
run "sed -i -E 's/(^|[^A-Za-z])TTS==0\\.21\\.1([^A-Za-z]|$)/TTS>=0.22,<0.23/g' requirements*.txt"

# 4b) не даём pip резолвить gradio заново
run "sed -i -E 's/^([[:space:]]*gradio[[:space:]]*==[[:space:]]*[0-9.]+[[:space:]]*)/# (preinstalled) \\1/' requirements_base.txt"
run "sed -i -E 's/^([[:space:]]*gradio[[:space:]]*==[[:space:]]*[0-9.]+[[:space:]]*)/# (preinstalled) \\1/' requirements_extra.txt || true"

# 4c) витрина для дебага
echo "=== REQUIREMENTS SNAPSHOT (after sed) ==="
grep -nE '(^|[[:space:]])(gradio|numpy|websockets|pyannote|docopt)|(^|[[:space:]])(-c|--constraint)[[:space:]]' requirements*.txt || true

# 5) установка зависимостей ПРОЕКТА под constraints
run "env -u PIP_ONLY_BINARY uv run python -m pip install -v -r requirements_base.txt  --constraint /content/constraints_sonitranslate.txt --log $LOG_DIR/pip_req_base.log"
run "env -u PIP_ONLY_BINARY uv run python -m pip install -v -r requirements_extra.txt --constraint /content/constraints_sonitranslate.txt --log $LOG_DIR/pip_req_extra.log || true"

# 6) sanity-импорты
python - << 'PY'
import sys, os, numpy, torch, gradio
print("python:", sys.version.split()[0])
print("torch:", torch.__version__, "| cuda_available:", torch.cuda.is_available())
print("numpy:", numpy.__version__)
print("gradio:", gradio.__version__)
print("PROFILE:", os.environ.get("PROFILE"))
PY

echo ""
echo "[ok] install test on CPU completed"
echo "Logs saved in: $LOG_DIR"





Python: Python 3.12.11
PIP_CONSTRAINT=<unset>
PROFILE=upstream
NUMPY_SPEC=numpy==1.26.4
GRADIO_SPEC=gradio==4.19.2
=== ACTIVE CONSTRAINTS FILE ===
numpy==1.26.4
scipy>=1.10
websockets>=10,<12
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.41
sentence-transformers>=3.0
gradio==4.19.2
== ENV ==
SONITRANSLATE: https://github.com/tekswirl25/SoniTranslate.git @ main
WHISPERX:      https://github.com/tekswirl25/whisperX.git @ cuda_12_x
PYANNOTE:      https://github.com/tekswirl25/pyannote-audio.git @ 3.1.1
LOG_DIR:       /content/_install_logs

+ python -m pip install -v --upgrade pip wheel uv --log /content/_install_logs/pip_bootstrap.log
[07:02:18] Using pip 25.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
[07:02:18] Requirement already satisfied: pip in /usr/local/lib/python3.12/dist-packages (25.2)
[07:02:18] Requirement already satisfied: wheel in /usr/local/lib/python3.12/dist-packages (0.45.1)
[07:02:18] Requirement already satisfied: uv in /usr/local/lib/

CalledProcessError: Command 'b'# [STEP 08/08] DRY INSTALL ON CPU (ISOLATED COPY, VERBOSE LOGS)\n# profile-driven; pins \xd1\x82\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe \xd0\xb8\xd0\xb7 07MODE/07AUTO/07CONS; \xd0\xb1\xd0\xb5\xd0\xb7 \xd1\x85\xd0\xb0\xd1\x80\xd0\xb4\xd0\xba\xd0\xbe\xd0\xb4\xd0\xbe\xd0\xb2 \xd0\xb7\xd0\xb4\xd0\xb5\xd1\x81\xd1\x8c\nset -euo pipefail\n\nLOG_DIR="/content/_install_logs"\nmkdir -p "$LOG_DIR"\n\necho "Python: $(python -V)"\necho "PIP_CONSTRAINT=${PIP_CONSTRAINT:-<unset>}"\necho "PROFILE=${PROFILE:-<unset>}"\necho "NUMPY_SPEC=${NUMPY_SPEC:-<unset>}"\necho "GRADIO_SPEC=${GRADIO_SPEC:-<unset>}"\necho "=== ACTIVE CONSTRAINTS FILE ==="\ntest -s /content/constraints_sonitranslate.txt || { echo "ERROR: /content/constraints_sonitranslate.txt missing"; exit 2; }\nsed -n \'1,120p\' /content/constraints_sonitranslate.txt\n\nexport PIP_DISABLE_PIP_VERSION_CHECK=1\nexport PIP_PROGRESS_BAR=on\nexport PIP_USE_PEP517=1\nexport PIP_PREFER_BINARY=1\nexport PYTHONUNBUFFERED=1   

In [ ]:
# @title Install requirements for SoniTranslate
!git clone https://github.com/r3gm/SoniTranslate.git
%cd SoniTranslate

!pip uninstall chex pandas-stubs ibis-framework albumentations albucore jax -y -q
!python -m pip install -q pip==23.1.2
!apt install git-lfs
!git lfs install

!sed -i 's|git+https://github.com/R3gm/whisperX.git@cuda_11_8|git+https://github.com/R3gm/whisperX.git@cuda_12_x|' requirements_base.txt
!sed -i 's|fairseq==0.12.2 |https://github.com/liyaodev/fairseq/releases/download/v0.12.3.1/fairseq-0.12.3.1-cp311-cp311-linux_x86_64.whl|' requirements_extra.txt
!pip install -q -r requirements_base.txt
!pip install -q -r requirements_extra.txt
!pip install -q ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/

Install_PIPER_TTS = True # @param {type:"boolean"}

if Install_PIPER_TTS:
    !pip install -q piper-tts==1.2.0

Install_Coqui_XTTS = True # @param {type:"boolean"}

if Install_Coqui_XTTS:
    !pip install -q -r requirements_xtts.txt
    !pip install -q TTS==0.21.1  --no-deps

!sudo apt install -y libcudnn8
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

One important step is to accept the license agreement for using Pyannote. You need to have an account on Hugging Face and `accept the license to use the models`: https://huggingface.co/pyannote/speaker-diarization and https://huggingface.co/pyannote/segmentation

Get your KEY TOKEN here: https://hf.co/settings/tokens

When you are creating the new Access Token in Hugging Face, make sure to tick "Read access to contents of all public gated repos you can access".

In [ ]:
#@markdown # `RUN THE WEB APP`
YOUR_HF_TOKEN = "" #@param {type:'string'}
%env YOUR_HF_TOKEN={YOUR_HF_TOKEN}
theme = "Taithrah/Minimal" # @param ["Taithrah/Minimal", "aliabid94/new-theme", "gstaff/xkcd", "ParityError/LimeFace", "abidlabs/pakistan", "rottenlittlecreature/Moon_Goblin", "ysharma/llamas", "gradio/dracula_revamped"]
interface_language = "english" # @param ['arabic', 'azerbaijani', 'chinese_zh_cn', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'japanese', 'korean', 'marathi', 'polish', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish', 'ukrainian', 'vietnamese']
verbosity_level = "info" # @param ["debug", "info", "warning", "error", "critical"]


%cd /content/SoniTranslate
!python app_rvc.py --theme {theme} --verbosity_level {verbosity_level} --language {interface_language} --public_url

Open the `public URL` when it appears